# Data Science Project Spring 2023

## 200+ Financial Indicators of US stocks (2014-2018)

### Yiwei Gong, Janice Herman, Alexander  Morawietz and Selina Waber

University of Zurich, Spring 2023

## Importing Packages

In [ ]:
import os 
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from pandas_datareader import data


from sklearn.model_selection import train_test_split

## Loading the Data Set


We used the data set from Nicolas Carbone from the webpage [kaggle](https://www.kaggle.com/datasets/cnic92/200-financial-indicators-of-us-stocks-20142018). Each dataset contains over 200 financial indicators, that are found in the [10-K filings](https://www.investopedia.com/terms/1/10-k.asp#:~:text=Key%20Takeaways-,A%2010%2DK%20is%20a%20comprehensive%20report%20filed%20annually%20by,detailed%20than%20the%20annual%20report.) of publicly traded companies from the US between the years 2014 - 2018.

In [ ]:
project_directory = sys.path[0] ## get path of project directory
data_directory = os.path.join(project_directory, 'data')

years = [2014, 2015, 2016, 2017, 2018]

## Loading the yearly dataset into the array dfs
dfs = []
for year in years:
    df = pd.read_csv(os.path.join(data_directory, f'{year}_Financial_Data.csv'), sep=',')
    df['year'] = np.full(df.shape[0], str(year)) ## append column with the year respecitvely
    df['PRICE VAR [%]'] = df[f'{year +1} PRICE VAR [%]'] ## Adding variable of the same name for all df, e.g. '2016 PRICE VAR [%]' renamed to 'PRICE VAR [%]'
    df = df.drop(columns=[f'{year +1} PRICE VAR [%]']) # dropp year-specific variable name
    df.columns.values[0] = 'Stock Name' # name the first variable 
    dfs.append(df)
    
    
df = pd.concat(dfs, ignore_index=True) ## concat the diffrent dataframes
df.head()

## Some Explanation of Variables:

### Adding  `year` as a cathegorical variable

We added a column named `year` which contains the respecitve year.



### Handling the variable `Price VAR [%]`

The last column, `PRICE VAR [%]`, lists the percent price variation of each stock for the year. For example, if we consider the dataset 2015_Financial_Data.csv, we will have:

- 200+ financial indicators for the year 2015;
- percent price variation for the year 2016 (meaning from the first trading day on Jan 2016 to the last trading day on Dec 2016).

We renamed all the variables with the specific year in it, e.g.  `2016 PRICE VAR [%]` to `PRICE VAR [%]`. We dropped the old ones.Now we just have one variable `PRICE VAR [%]`. 


### The variable `class`

class lists a binary classification for each stock, where

- for each stock, if the PRICE VAR [%] value is positive, class = 1. From a trading perspective, the 1 identifies those stocks that an hypothetical trader should BUY at the start of the year and sell at the end of the year for a profit.
- for each stock, if the PRICE VAR [%] value is negative, class = 0. From a trading perspective, the 0 identifies those stocks that an hypothetical trader should NOT BUY, since their value will decrease, meaning a loss of capital.


The columns `PRICE VAR [%]` and `class` make possible to use the datasets for both classification and regression tasks:

- If the user wishes to train a machine learning model so that it learns to classify those stocks that in buy-worthy and not buy-worthy, it is possible to get the targets from the class column;
- If the user wishes to train a machine learning model so that it learns to predict the future value of a stock, it is possible to get the targets from the PRICE VAR [%] column.


### The variable  `Stock Name`

We named the first variable `Stock Name`since it has not been named in the original dataset.



## First Description of the Data

In [ ]:
df.info(verbose=True)

## Numerical and Catgorical Features/Variables

We are converting `Class`to a cathegorical variable.

In [ ]:
df['Class'] = df.Class.astype('object') ## object or catheogry?? whats the difference???

numCols = df.select_dtypes(exclude='object').columns
print(f"There are {len(numCols)} numerical features:\n")

catCols = df.select_dtypes(include='object').columns
print(f"There are {len(catCols)} categorical features:\n", catCols)

## Any Duplicates? 

No, there are no duplicates for rows but there are 20 duplicates for columns/ 10 each. Not same variable name but same data!

In [ ]:
print(f'Duplicates in Rows:', True in list(df.duplicated()))


In [ ]:
print(f'Duplicates in Columns:', True in list(df.T.duplicated().T))
print("Show the Duplicates:")
print(df.T[df.T.duplicated(keep=False)].T)

Our Duplicates are the following pairs:

- `ebitperRevenue` and `eBITperRevenu`
- `ebtperEBIT` and `eBTperEBIT`
- `niperEBT` and `nIperEBT`
- `returnOnAssets` and `Return on Tangible Assets`
- `returnOnCapitalEmployed` and `ROIC`
- `payablesTurnover` and `Payables Turnover`
- `inventoryTurnover` and `Inventory Turnover`
- `debtRatio` and `Debt to Assets`
- `debtEquityRatio` and `Debt to Equity`
- `cashFlowToDebtRatio` and `cashFlowCoverageRatios`

We will remove the first occurence of the duplicates respectively.

In [ ]:
shape_old=df.shape

## HUGHE PROBLEM --> df.drop_duplicates() transforms all variables to the datatype 'object'!
#I don't know why it does that!
# So i will remove the manually!!!

#df= df.T.drop_duplicates().T # remove duplicates!

df= df.drop(columns=['eBITperRevenue', 'eBTperEBIT', 'nIperEBT', 'Return on Tangible Assets', 
                     'ROIC', 'Payables Turnover', 'Inventory Turnover', 'Debt to Assets', 'Debt to Equity', 
                     'cashFlowCoverageRatios'])

print(f' Shape with duplicates:', shape_old) 
print(f' Shape after removal of duplicates:', df.shape) 


#print(df.info(verbose=True)) ok good, sill folat64 objects


## Variation 
There is something seriously wrong with the variable `operatingProfitMargin`. It has a Standard Deviation of 0, which cant be! So we dropp it.

In [ ]:
df.std(axis=0, skipna=True, numeric_only=True)

no_std= [ std for std in df.std( axis=0, skipna=True, numeric_only=True) if std <= 0.15]
print(no_std)



#x = df.std( axis = 0, skipna = True, numeric_only = True)
#[(index, x[index]) for index in x.index.values if x[index] > 0]

### TO DOOO!!!!!!!!!!!!!!!!!

df= df.drop(columns= ['operatingProfitMargin'])


Furthermore, there are two variables (operatingCycle and cashConversionCycle) which have missing values in more than 99% of the time. These two columns are also dropped.

In [ ]:
most_nan_columns = df.isnull().mean(axis = 0)

print("Variables with maximal missing rate:")
print(most_nan_columns[most_nan_columns > 0.5])
df = df.drop(columns = ["operatingCycle", "cashConversionCycle"])

## Correlation of the variables

Although we removed all doublicates there are still many variables that are highly correlated. We consider variables with correlations larger than 0.98 as doublicates and remove them.

In [ ]:
#check_corr contains the predictor variables which have to be checked for high correlations.
check_corr = df.drop(labels=["Class", "Stock Name", "Sector", "year", "PRICE VAR [%]"], axis = 1)

In [ ]:
plt.matshow(check_corr.corr().abs())
plt.title("Correlations between the predictors")
plt.colorbar()

In [ ]:
abs_corr = check_corr.corr().abs()
for i in range(len(abs_corr)):
    abs_corr.iloc[i, i] = 0
    
    
abs_corr_unstack = abs_corr.unstack()
print("Variable pairs with high correlation")
print(abs_corr_unstack.sort_values(kind="quicksort")[-50:])

In [ ]:
#A variables is removed if the correlation with another variable is higher than 0.98.

columns_to_drop = []
columns_to_remain = []

for pair in abs_corr_unstack.index.values:
    if abs_corr_unstack[pair] > 0.98:
        if pair[0] not in columns_to_remain and pair[1] not in columns_to_remain:
                columns_to_remain.append(pair[0])
                if pair[1] not in columns_to_drop:
                    columns_to_drop.append(pair[1])
        elif pair[0] in columns_to_remain:
            if pair[1] not in columns_to_drop:
                columns_to_drop.append(pair[1])
        elif pair[1] in columns_to_remain:
            if pair[0] not in columns_to_drop:
                columns_to_drop.append(pair[0])

df = df.drop(columns=columns_to_drop)

In [ ]:
print(f"{len(columns_to_drop)} variables were removed as dublicates.")

## Class Balance

The Variable `Class`is not balanced. We have to keep that in mind for train and test split. 

In [ ]:
print(f"The class variable is 1 in {df['Class'].value_counts()[1]} cases.")
print(f"The class variable is 0 in  {df['Class'].value_counts()[0]} cases.")

## Description of Outliers

In our data set, we observe several columns containing values that are multiple orders larger than the 75% percentil or smaller than the 25% percentil. For example, the maximum value of the variable "revenue" is 1.9e12 while the 75% percentil is 2.3e9. However, it is possible that a large company has an exceding revenue compared to others. Therefore, it is not trivial to decide if such points are errors or reflect a true value.

As shown below, several outliers are observed in the variable "PRICE VAR [%]". As the dependet variable, its outliers are likey to strongly bias the statistical models. Therefore, the variable "PRICE VAR [%]" is analysed in more detail.

In [ ]:
df.describe()

In [ ]:
df_ = df.loc[:, ['Sector','PRICE VAR [%]']]

# Get list of sectors
sector_list = df_['Sector'].unique()

# Plot the percent price variation for each sector
for sector in sector_list:
    
    temp = df_[df_['Sector'] == sector]

    plt.figure(figsize=(30,5))
    plt.plot(temp['PRICE VAR [%]'])
    plt.title(sector.upper())
    plt.show()
    

In [ ]:
# Get stocks that increased more than 500%
gain = 500
top_gainers = df[df['PRICE VAR [%]'] >= gain]
top_gainers = top_gainers['PRICE VAR [%]'].sort_values(ascending=False)
print(f'{len(top_gainers)} STOCKS with more than {gain}% gain.')

## Cleaning of Outliers
As described above, our data set contains many outliers which could reflect wrong data point, e.g. due to mistyping. Since some of our models are sensitive to outliers, we decided to replace the most extreme 2% of our data with the 0.99 quantile or with the 0.01 quantile, respectively. 

In [ ]:
top_quantiles = df.quantile(0.99)
outliers_top = (df > top_quantiles)

low_quantiles = df.quantile(0.01)
outliers_low = (df < low_quantiles)

df = df.mask(outliers_top, top_quantiles, axis=1)
df = df.mask(outliers_low, low_quantiles, axis=1)

In [ ]:
df.describe()

In [ ]:
## Z-Score: 

threshold = 3

test= df[['ebtperEBIT', 'returnOnCapitalEmployed']]
for col in test:
    z_score= stats.zscore(df[col], nan_policy='omit')
    outlier_indices = np.where(z_score > threshold)
    print(outlier_indices )


In [ ]:
# IQR
test= df[['ebtperEBIT', 'returnOnCapitalEmployed']]
for col in test:
    iqr= stats.iqr(df[col], nan_policy='omit')
    print(iqr)

#''''''

https://www.kaggle.com/code/nareshbhat/outlier-the-silent-killer

## Description of Missing Data

In our data set, we observe many missing values. There are several columns with more than 30% of NaN values. In the following, a short description of these columns is given.

In [ ]:
print(f'There are in total {df.isnull().sum().sum()} NAN in the dataframe')

In [ ]:
#df.replace([np.inf, -np.inf], np.nan, inplace=True) do that in Outliers Section!
#df.replace(to_replace = 0, value = np.nan, inplace=true) ????? DA EVTL noch mehr mit NAN ersetzen!

In [ ]:
## Overview of all variables with missing values
df.isnull().mean().sort_values(ascending=False).plot.bar(figsize=(100,20))
plt.ylabel('Percentage of missing values')
plt.xlabel('Variables')
plt.title('Quantifying ALL missing data')

In [ ]:
most_nan = df.isnull().mean().sort_values(ascending=False)
most_nan = most_nan[most_nan > 0.3]

most_nan.plot.bar(figsize=(8,4))
plt.ylabel('Percentage of missing values')
plt.title('Columns with more than 30% missing values')


In [ ]:
# Percentage of missing values for the variables
missing = df.isnull().sum().sort_values(ascending=False)
percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([missing, percent], axis=1, keys=['Nr. of missing values', 'Percent of Missing Values'])
missing_data.head(25)

In [ ]:
# Plot missing values 2.0
sns.heatmap(df.isna().transpose(), cmap="Blues", cbar_kws={'label': 'Missing Values'});

It is conspicuous that there are columns with more than 50% of the values equal to zero. It is likely that these cases reflect missing values. Therefore, we decided to consider them as missing values.

In [ ]:
most_zero = df.drop(columns = "Class").isin([0]).mean().sort_values(ascending=False)
most_zero = most_zero[most_zero > 0.5]

most_zero.plot.bar(figsize=(8,4))
plt.ylabel('Percentage of missing values')
plt.title('Columns with more than 50% zero values')

## Cleaning of Missing Data

To remove the missing values, we decided to drop all columns with more than 30% of NaNs. Furthermore, we dropped all columns with more than 50% of zero values. The remaining NaN fields are replaced by the median of each column.

In [ ]:
#Drop all columns with nr. of NaN > 30%

print("Shape of df data:", df.shape)

df = df.drop(columns=most_nan.index.values)

print("Shape of df data after removing columns with nr. of NaN > 30%:", df.shape)

df = df.drop(columns=most_zero.index.values)

print("Shape of df data after removing columns with nr. of zeros > 50%:", df.shape)

In [ ]:
numCols = df.select_dtypes(include=['float64', 'int64']).columns
print("New numerical columns:", numCols)
df[numCols] = df[numCols].fillna(df[numCols].median())

catCols = df.select_dtypes(exclude=np.number).columns
print("New categorical columns:", catCols)
for col in catCols:
    df[col].fillna("Unknown", inplace=True)
    
df.head()

In [ ]:
# Checking Missing data again
missing = df.isnull().sum().sort_values(ascending=False)
percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([missing, percent], axis=1, keys=['Nr. of missing values', 'Percent of Missing Values'])
missing_data.head(25)

## Adding Dummies

In [ ]:
# Factorize categorical values, assign output to X

# create (multiple) dummy variables for a categorical variable
df = pd.get_dummies(df, columns=catCols.drop("Stock Name"))
df.head()

## Assigning X and target y

In [ ]:
X = df.drop(labels=["Class_0", "Class_1", "Stock Name", "year_2014", "year_2015", "year_2016", "year_2017", "year_2018", "PRICE VAR [%]"], axis = 1)
y = df["Class_1"] 

## Train and Test Split

In [ ]:
## Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, test_size = 0.3, random_state = 42) 

df_train = pd.concat([X_train, y_train], axis=1)
df_test = pd.concat([X_test, y_test], axis=1)

print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")

## Important Link

[Cleaning Data Set](https://www.kaggle.com/code/cnic92/explore-and-clean-financial-indicators-dataset)

## Feature Selection

As a last step in the exploratory data analysis we will use an Extra Trees Classifier to extract the most important features of our data set. This feature selection method will be included later in our pipeline. At this point, we want to anaylse the most important features in more detail.

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

In [ ]:
# Selecting most significant features

# Feature selection using Extra Trees Classifier on the resampled training data
model = ExtraTreesClassifier(random_state=42)
model.fit(X, y)
importances = model.feature_importances_

# Select top features with highest importance scores
top_features = pd.Series(importances, index=X.columns).nlargest(300)

In [ ]:
# plot the importance of all features
n = len(top_features)

# Plot feature importance of all features
plt.figure(figsize=(12, 8))
plt.bar(range(n), top_features[:n], align='center')

plt.xlim([-1, n])
plt.xlabel('Feature')
plt.ylabel('Feature Importance')

plt.tight_layout()

In [ ]:
# plot the importance of all features
n = 15

plt.figure(figsize=(10, 8))

# Plot the most important features
plt.subplot(1,2,1)
plt.bar(range(n), top_features[:n], align='center')

for i in range(n):
    plt.text(i, top_features[i]+1e-4 , round(top_features[i], 3), ha='center', fontsize = 6)

plt.xticks(range(n), top_features.index[:n], rotation=90)
plt.xlim([-1, n])
plt.ylim([0, 0.015])
plt.title(f"The {n} most important features")
plt.ylabel('Feature Importance')

# Plot the 10 least important features

plt.subplot(1,2,2)
plt.bar(range(n), top_features[-n:], align='center')

for i in range(n):
    plt.text(i, top_features[len(top_features)-n+i]+1e-4 , round(top_features[len(top_features)-n+i], 3), ha='center', fontsize = 6)

plt.xticks(range(n), top_features.index[-n:], rotation=90)
plt.xlim([-1, n])
plt.ylim([0, 0.01])
plt.title(f"The {n} least important features")
plt.ylabel('Feature Importance')

plt.tight_layout()

It is interesting to note that the least important feature is the categorial variable "Sector". Furthermore, it is not surprising that the most important features are EPS ("earnings per share") or revenue growth (either averaged over one, three or five years) which are closely connected with the price variance of the share. However, many variables measure almost identical finacial indicators and are therefore highly correlated. Therefore, the importances are similar and a choice of a subset of featured for a reduced data set is rather arbitrary. Nevertheless, a description of the 10 most important variables is given in the following.

In [ ]:
#Scatter_matrix of the 10 most important features

n=10

plt.figure(figsize = (20,20))
axes = pd.plotting.scatter_matrix(X[top_features.index[:n]], alpha=0.2)
for ax in axes.flatten():
    #ax.xaxis.label.set_rotation(90)
    ax.set_xlabel("")
    ax.yaxis.label.set_rotation(0)
    ax.yaxis.label.set_ha('right')

plt.tight_layout()
plt.gcf().subplots_adjust(wspace=0, hspace=0)
plt.show()

In [ ]:
#dependence between the most important variable and PRICE VAR [%]

plt.scatter(X[top_features.index[0]], df["PRICE VAR [%]"])
plt.xlabel(top_features.index[0])
plt.ylabel("PRICE VAR [%]")

In [ ]:
fig, axes = plt.subplots(5,2)
fig.set_size_inches(20, 20)

for i, ax in enumerate(axes.flatten()):
    feat_cl0 = df[y == 0][top_features.index[i]]
    feat_cl1 = df[y == 1][top_features.index[i]]
    
    mean_cl0 = feature_class0.mean()
    mean_cl1 = feature_class1.mean()
    
    med_cl0 = feature_class0.median()
    med_cl1 = feature_class1.median()    
    
    plot_range = (feat_cl0.quantile(0.05), feat_cl0.quantile(0.95))
    ax.hist(feat_cl0, color = "red", bins = 100, alpha = 0.5, range = plot_range, label = "Sell")
    ax.axvline(x = med_cl0, color = "red", linestyle='--', label = f"median {round(med_cl0, 3)}")
    ax.axvline(x = mean_cl0, color = "red", linestyle=':', label = f"mean {round(mean_cl0, 3)}")
    ax.hist(feat_cl1, color = "green", bins = 100, alpha = 0.5, range = plot_range, label = "Buy")
    ax.axvline(x = med_cl1, color = "green", linestyle='--', label = f"mean {round(med_cl1, 3)}")
    ax.axvline(x = mean_cl1, color = "green", linestyle=':', label = f"median {round(med_cl1, 3)}")
    ax.set_xlabel(top_features.index[i])
    ax.legend()

plt.tight_layout()
plt.show()

## Neural Network

In [ ]:
import torch
from torchvision import datasets as datasets
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import torchvision

from skorch import NeuralNetClassifier

from sklearn.model_selection import GridSearchCV

from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay, classification_report)

import seaborn as sn

The predictor variables for the neural network include all the numerical variables and the categorial dummy variables "Sector_x". The numerical variables are standardized by the z-score standardization. The network is trained on the binary variable "Class". 

In [ ]:
X_std = X.copy()
stdCols = X_std.select_dtypes(include = "float64").columns #columns to be standardized
X_std[stdCols] = (X_std[stdCols]-X_std[stdCols].mean())/X[stdCols].std()

### FCNetwork

We build a fully connected network with two hidden layers. To prevent overfitting a dropout layer follows both the hidden layers. We use the rectified linear unit function for activation. The output is transformed by the sigmoid function to find the predicted values between 0 and 1.

In [ ]:
input_dim = X.shape[1]
batch_size = 4

In [ ]:
class FCNetwork(nn.Module):
    def __init__(self, dropout_rate=0.3, input_dim = input_dim):
        super().__init__()
        self.lin1 = nn.Linear(input_dim, input_dim//2)
        self.lin2 = nn.Linear(input_dim//2, input_dim//4)
        self.lin3 = nn.Linear(input_dim//4, 1)
        self.drop1 = nn.Dropout(p = dropout_rate)
        self.drop2 = nn.Dropout(p = dropout_rate)
        self.prob = nn.Sigmoid()

    def forward(self, x):
        x = F.relu(self.lin1(x))
        x = self.drop1(x)
        x = F.relu(self.lin2(x))
        x = self.drop2(x)
        x = self.lin3(x)
        x = self.prob(x)
        return x

### Grid Search for optimal Hyperparameters
To improve the accuracy of the model, the hyperparameters "learning rate", "momentum" and "dropout rate" are optimized by grid search. This is performed by the function Gridsearch which uses GridSearchCV from sklearn.model_selection and returns the optimal parameters from the grid "param_grid".


In [ ]:
#Transformation of the data to tensors for the Grid Search
X_tensor = torch.tensor(X.values, dtype=torch.float32)
y_tensor = torch.tensor(y.astype("int32").values, dtype=torch.float32).reshape(-1, 1)

In [ ]:
param_grid = {"optimizer__lr": [0.001, 0.004, 0.007, 0.01], 
              "optimizer__momentum": [0.6, 0.7, 0.8, 0.9],
              "module__dropout_rate": [0.2, 0.3, 0.4, 0.5]}

In [ ]:
def GridSearch(X, y, param_grid, max_epochs, batch_size):
    #Transformation of the data to tensors for the Grid Search
    X_tensor = torch.tensor(X.values, dtype=torch.float32)
    y_tensor = torch.tensor(y.astype("int32").values, dtype=torch.float32).reshape(-1, 1)
    
    model = NeuralNetClassifier(
    FCNetwork,
    criterion=nn.BCELoss,
    optimizer=optim.SGD,
    max_epochs=max_epochs,
    batch_size=batch_size,
    verbose=False
    )
    
    grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs = -1)
    grid_result = grid.fit(X_tensor, y_tensor)
    
    print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
    means = grid_result.cv_results_['mean_test_score']
    stds = grid_result.cv_results_['std_test_score']
    params = grid_result.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params):
        print("%f (%f) with: %r" % (mean, stdev, param))
        
    return grid_result.best_params_

In [ ]:
max_epochs = 5
optimal_hyperp = GridSearch(X, y, param_grid, max_epochs, batch_size)

### Training the Neural Network with the optimal parameters
The optimal hyperparameters were found to be:
    learning rate = 0.01,
    momentum = 0.9,
    dropout rate = 0.2.
   
With these hyperparameters and a batch size of 4 the neural network is trained using pytorch. For this purpose, a class CustomDataset is created which transforms the data to torch.tensors and contains a method __get_item__(self, idx) for the data loader. The dataloader is defined in the function CustomDataloader(X, y). It splits the data set in parts for training, evaluation and testing. By using the class DataLoader from torch.utils.data it returns the data set in the appropriate data type for the FCNetwork. The training is performed by the function FCNetwork_train(network, trainloader, valloader, PATH, nr_epochs, print_running_loss) which saves the optimal network parameters in a file under PATH. The function test_true_predicted(network, testloader, PATH): returns a list of the true class labels and a list of the predicted class labels from the test set.

In [ ]:
optimal_lr = 0.01 #GridSearch(X, y, param_grid, max_epochs, batch_size)["optimizer__lr"]
optimal_momentum = 0.9 #GridSearch(X, y, param_grid, max_epochs, batch_size)["optimizer__momentum"]
optimal_dropout = 0.2 #GridSearch(X, y, param_grid, max_epochs, batch_size)["module__dropout_rate"]

In [ ]:
#Definition of Dataset for Dataloader
class CustomDataset(Dataset):
    def __init__(self, X, y):
        super().__init__()
        self.X = torch.tensor(X.values, dtype=torch.float32)
        self.y = torch.tensor(y.astype("int32").values, dtype=torch.float32).reshape(-1, 1)
        
    def __len__(self):
        return len(self.X)
        
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [ ]:
def CustomDataloader(X, y):    
    X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, test_size = 0.3, random_state = 42) 
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, stratify = y_train, test_size = 0.2, random_state = 42)

    train_dataset = CustomDataset(X_train, y_train)
    val_dataset = CustomDataset(X_val, y_val)
    test_dataset = CustomDataset(X_test, y_test)

    trainloader = DataLoader(train_dataset, batch_size=batch_size,
                                              shuffle=True)

    valloader = DataLoader(val_dataset, batch_size=batch_size,
                                             shuffle=False)

    testloader = DataLoader(test_dataset, batch_size=batch_size,
                                             shuffle=False)
    
    return trainloader, valloader, testloader

In [ ]:
def FCNetwork_train(network, trainloader, valloader, PATH, nr_epochs, print_running_loss):
    criterion = nn.BCELoss()
    optimizer = optim.SGD(network.parameters(), lr=optimal_lr, momentum=optimal_momentum)
    
    min_val_loss = float("inf")
    for epoch in range(nr_epochs):  

        running_loss = 0.0
        network.train()
        n = 0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data

            optimizer.zero_grad()

            outputs = network(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            n += 1

        if print_running_loss:
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / n:.3f}')

        network.eval()
        val_loss = 0
        n = 0
        with torch.no_grad():
            for i, data in enumerate(valloader, 0):
                inputs, labels = data

                outputs = network(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                n += 1

        if print_running_loss:
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {val_loss / n:.3f}')            

        if val_loss < min_val_loss:
            min_val_loss = val_loss
            torch.save(network.state_dict(), PATH)
            
            if print_running_loss:   
                print(f"The new best model is at epoch {epoch}")
        
        if print_running_loss:
            print(f'Epoch: {epoch} over')

In [ ]:
def test_true_predicted(network, testloader, PATH): #, print_accuracy):
    network.load_state_dict(torch.load(PATH))
    
    #correct = 0
    #total = 0
    
    y_pred = []
    y_true = []
    
    network.eval()
    with torch.no_grad():
        for data in testloader:
            inputs, labels = data
            outputs = network(inputs)
            predicted = outputs>0.5 
            
            y_true.extend(torch.reshape(labels, (batch_size, )).tolist())
            y_pred.extend(torch.reshape(predicted, (batch_size, )).tolist())
            
            #total += labels.size(0)
            #correct += (predicted == labels).sum().item()
    
    #if print_accuracy:
    #print(f'Accuracy of the network on the test images: {100 * correct / total} %')
    return y_true, y_pred #correct / total, 

In [ ]:
PATH = './net.pth'

In [ ]:
network = FCNetwork(input_dim = input_dim, dropout_rate = optimal_dropout)
trainloader, valloader, testloader = CustomDataloader(X_std, y)
FCNetwork_train(network, trainloader, valloader, PATH, nr_epochs=10, print_running_loss=False)

In [ ]:
y_true = test_true_predicted(network, testloader, PATH)[0]
y_pred = test_true_predicted(network, testloader, PATH)[1]

acc = accuracy_score(y_true, y_pred)

print("Full model", acc)

In [ ]:
cf_matrix = confusion_matrix(y_true, y_pred)
plt.figure(figsize = (8,5))
sn.heatmap(cf_matrix/len(y_true), annot=True)

To analyze the significance of the different predictor variables, we train different models, omitting one of the predictor variables at a time. While the accuracy of the model trained on the full data set is approximately 0.59, the accuracy of several models trained on a reduced data set drops below 0.55.

In [ ]:
accuracy_dropped_variable = []

for column in X_std.columns.values:
    
    network = FCNetwork(input_dim = input_dim-1, dropout_rate = optimal_dropout)
    
    trainloader, valloader, testloader = CustomDataloader(X_std.drop(columns = column), y)

    FCNetwork_train(network, trainloader, valloader, PATH, nr_epochs=10, print_running_loss=False)
        
    y_true = test_true_predicted(network, testloader, PATH)[0]
    y_predicted = test_true_predicted(network, testloader, PATH)[1]

    acc = accuracy_score(y_true, y_predicted)
   
    accuracy_dropped_variable.append(acc)
    
    print(column, acc)            

In [ ]:
accuracy_dropped_variable = pd.Series(accuracy_dropped_variable, index = X.columns.values)
accuracy_dropped_variable = accuracy_dropped_variable.sort_values(axis = 0)
accuracy_dropped_variable.head(10)

In [ ]:
plt.figure(figsize = (25,10))
barplot = plt.bar(influence.index.values, influence)
xticks = plt.xticks(influence.index.values, rotation='vertical', fontsize=5)
plt.ylim((0.54, 0.6))
plt.plot([0.58 for i in range(len(accuracy_with_dropped_variable))], linestyle='dashed', color = "black")
plt.ylabel("accuracy")
plt.xlabel("name of dropped variable")

In [ ]:
most_influencial_variables = accuracy_dropped_variable[accuracy_dropped_variable < 0.58].index.values

## Testing Random Forest

In [ ]:
from sklearn.ensemble import (RandomForestClassifier, ExtraTreesClassifier)
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay, classification_report)

In [ ]:
# Selecting most significant features

# Feature selection using Extra Trees Classifier on the resampled training data
model = ExtraTreesClassifier(random_state=42)
model.fit(X_train, y_train)
importances = model.feature_importances_

# Select top features with highest importance scores
top_features = pd.Series(importances, index=X_train.columns).nlargest(300)

# Subset X_resampled and X_test with selected features
X_train_selected = X_train[top_features.index]
X_test_selected = X_test[top_features.index]

In [ ]:
# Random Forest Classifier with the best hyperparameters found
rf_best = RandomForestClassifier(
    n_estimators=1670,
    criterion='gini',
    max_depth=18,
    min_samples_split=3,
    min_samples_leaf=1,
    min_weight_fraction_leaf=0.0,
    max_features='sqrt',
    max_leaf_nodes=None,
    min_impurity_decrease=0.0,
    bootstrap=False,
    oob_score=False,
    n_jobs=-1,
    class_weight='balanced',
    random_state=42
)

# Train the Random Forest model with the best hyperparameters
rf_best.fit(X_train_selected, y_train)

# Predict the test set
y_pred_rf = rf_best.predict(X_test_selected)

# Model evaluation
acc_rf = accuracy_score(y_test, y_pred_rf)
prec_rf = precision_score(y_test, y_pred_rf, average='macro')
rec_rf = recall_score(y_test, y_pred_rf, average='macro')
f1_rf = f1_score(y_test, y_pred_rf, average='macro')
print("Random Forest Classifier: Accuracy = %.3f, Precision = %.3f, Recall = %.3f, F1 Score = %.3f" % (acc_rf, prec_rf, rec_rf, f1_rf))

In [ ]:
print(y_test)
print(y_pred_rf)

## Test SVM

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.svm import SVC

# Create StandardScaler object
sc = StandardScaler()

# Standardize features; equal results as if done in two
X_train_std = sc.fit_transform(X_train_selected)
X_test_std = sc.transform(X_test_selected)

In [ ]:
# Grid Search for getting optimal C and gamma
gamma_range = np.outer(np.logspace(-3, 0, 4),np.array([1,5]))
gamma_range = gamma_range.flatten()
print(gamma_range)

C_range = np.outer(np.logspace(-1, 1, 3),np.array([1,5]))
C_range = C_range.flatten()
print(C_range)

parameters = {'kernel':['linear', 'rbf'], 'C':C_range, 'gamma': gamma_range}

svm = SVC()
grid = RandomizedSearchCV(estimator=svm, param_distributions=parameters, n_iter=5, n_jobs=-1, verbose=2)
grid.fit(X_train_std, y_train)

print('Best CV accuracy: {:.2f}'.format(grid.best_score_))
print('Test score:       {:.2f}'.format(grid.score(X_test_std, y_test)))
print('Best parameters: {}'.format(grid.best_params_))